## Connect to Google drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## read in Train and Test data directly from csv

In [ ]:
# Read in CSV data
import pandas as pd
train_df = pd.read_csv("/content/gdrive/MyDrive/W266 Final Project/data/train_data.csv")
train_df_filter = train_df[['judgement','summary']]
test_df = pd.read_csv("/content/gdrive/MyDrive/W266 Final Project/data/test_data.csv")
test_df_filter = test_df[['judgement','summary']]

## extractive models first

In [ ]:
!pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 21.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21696 sha256=0dbc21de30efaec3bc9b941cd6ba9979123e138ad7a57eff0e07186dc55fe5c3
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=51f050d56b499859615af64c003a68308783f683969a121d0c25f5f315619cbd
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=1068

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

# Initialize the summarizer with the TextRank algorithm
summarizer = LsaSummarizer()

baseline_output = []

for judgement in train_df_filter['judgement'][:5]:
  parser = PlaintextParser.from_string(judgement, Tokenizer("english"))

  # Summarize the article and get the most important sentences
  summary = summarizer(parser.document, 100)  # You can change the number of sentences as needed
  summary_sentences = [str(sentence) for sentence in summary]
  baseline_output.append(summary_sentences)

len(baseline_output)

5

In [2]:
import pandas as pd
train_data_original = pd.read_csv("/content/gdrive/MyDrive/W266 Final Project/data/train_data.csv")

In [2]:
import pandas as pd
extractive_output = pd.read_csv("/content/gdrive/MyDrive/W266 Final Project/output/train_data_1000.csv")

## Abstractive model

In [3]:

!pip install -q transformers
!pip install -q rouge_score
!pip install -q evaluate
import evaluate
#let's make longer output readable without horizontal scrolling
from pprint import pprint


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00


In [ ]:
len(extractive_output["Summary"][207].split())

868

In [ ]:
# Assuming 'extractive_output["Summary"]' is the list containing summaries

# Step 1: Calculate word count for each record in the column
word_counts = []

for summary in extractive_output["Summary"]:
    if isinstance(summary, str):  # Check if the summary is a string
        word_counts.append(len(summary.split()))

# Step 2: Sum all the word counts
total_word_count = sum(word_counts)

# Step 3: Divide the sum by the total number of valid records to get the average word count
if len(word_counts) > 0:
    average_word_count = total_word_count / len(word_counts)
    print("Average word count:", average_word_count)
else:
    print("No valid summaries found.")


Average word count: 863.0986918792903


In [ ]:
extractive_output['Index'][0]

'uksc-2009-0034.txt'

In [ ]:
len(extractive_output)

1000

### LongT5

In [4]:
from transformers import AutoTokenizer, LongT5ForConditionalGeneration

model = (
    LongT5ForConditionalGeneration.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")
    .to("cuda")
    .half()
)
tokenizer = AutoTokenizer.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")

In [ ]:
ARTICLE_TO_SUMMARIZE = "summarize: " + extractive_output["ExtractiveSummary"][0]
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=16384, return_tensors="pt", truncation=True)
input_ids = inputs.input_ids.to("cuda")
attention_mask = inputs.attention_mask.to("cuda")

outputs = model.generate(input_ids, attention_mask=attention_mask, num_beams=3, min_length=512, max_length=1024, temperature=1.0, repetition_penalty=2.5, length_penalty=2.0)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
pprint(summary, compact = True)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:835: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


('this article describes the case of a woman who had been married for 38 years '
 'prior to their divorce in 2005 on the husbands petition in Nigeria. she was '
 'diagnosed as having a foreign divorce by means of an ex parte application by '
 'the wife for leave to make an application under part 3 and confirmed his '
 'decision on the husbands application to set it aside. on the substantive '
 'hearing colridgej made an order which was intended to enable the wife to '
 'house and maintain herself inLondon by providing her 65% of the proceeds of '
 'sale ( expected to be about 275,000 ) of the house in which she has been '
 'living. the court of appeal allowed the husbands appeal principally on the '
 'ground that the judge had given insufficient weight to the connections of '
 'the case with the case with Nigeria ( [ 2009]eWCa citv 1, [2009] 3wl835 ). '
 'the court of Appeal allowed the husbands appeal principally on the ground '
 'that the judge had given insufficient weight to the co

In [ ]:
# Creating a dictionary to represent the data
data = {
    'ExpectedSummary': [summary],
    'PredictedSummary': [extractive_output["Summary"][0]]
}

# Creating the DataFrame
df = pd.DataFrame(data)


In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=df['PredictedSummary'],
                        references=df['ExpectedSummary'])
print(results)

{'rouge1': 0.4401064773735582, 'rouge2': 0.12622222222222224, 'rougeL': 0.21118012422360247, 'rougeLsum': 0.2803904170363798}


### fine tuning

In [5]:
example_df = extractive_output.head()
for index, row in example_df.iterrows():
    name = row['Index']
    print(f"Index name: {name}")

Index name: 4809.txt
Index name: 543.txt
Index name: 372.txt
Index name: uksc-2010-0177.txt
Index name: 6391.txt


In [ ]:
example_df

,Index,Summary,ExtractiveSummary
0,4809.txt,The first respondent joined M.B.B.S. course of...,Civil Appeal No. 2828 of 1977. Appeal by Speci...
1,543.txt,"On the death of R, a Hindu jat, in April or Ma...",Civil Appeal No. 137 of 1953. Appeal from the ...
2,372.txt,An application was filed by the first responde...,Appeal No. 312 of 1955. On appeal by special l...
3,uksc-2010-0177.txt,Scottish Widows Plc (Scottish Widows) is a lif...,This is an appeal from an interlocutor of the ...
4,6391.txt,The appellant and the respondents applied for ...,vil Appeal Nos. 16 16 17 of 1990. From the Jud...


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")

In [ ]:
max_input_length = 16368
max_target_length = 1024
prefix = "summarize: "

def preprocess_function(example):
    inputs = [prefix + doc for doc in example["ExtractiveSummary"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, return_tensors="pt", truncation=True, padding=True)
    labels = tokenizer(text_target=example["Summary"], max_length=max_target_length, truncation = True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = example_df.apply(preprocess_function, axis=1)

In [ ]:
tokenized_datasets[0]['input_ids']

tensor([[21603,    10,   205,     1,     0],
        [21603,    10,     3,    23,     1],
        [21603,    10,     3,   208,     1],
        ...,
        [21603,    10,     3,    15,     1],
        [21603,    10,     3,    26,     1],
        [21603,    10,     3,     5,     1]])

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model_checkpoint = "Stancld/longt5-tglobal-large-16384-pubmed-3k_steps"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

### Run for the whole 1000 records:

In [ ]:
# Assuming you have already defined the ARTICLE_TO_SUMMARIZE, tokenizer, model, and evaluate objects.

# Initialize lists to store the summaries and expected summaries
predicted_summaries = []
expected_summaries = []
document_names = []

for index, row in extractive_output.iterrows():
    name = row['Index']
    original_summary = row['Summary']
    extractive_summary = row['ExtractiveSummary']

    ARTICLE_TO_SUMMARIZE = "summarize: " + extractive_summary

    # Tokenize the article
    inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=16384, return_tensors="pt", truncation=True)
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    # Generate the summary
    outputs = model.generate(input_ids, attention_mask=attention_mask, num_beams=3, min_length=512, max_length=1024, temperature=1.0, repetition_penalty=2.5, length_penalty=2.0)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Append the predicted and expected summaries to their respective lists
    predicted_summaries.append(summary)
    expected_summaries.append(original_summary)
    document_names.append(name)

# Creating a dictionary to represent the data
data = {
    'DocumentName' : document_names,
    'ExpectedSummary': expected_summaries,
    'PredictedSummary': predicted_summaries
}

# Creating the DataFrame
df = pd.DataFrame(data)

# Perform Rouge evaluation on the whole dataset
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=df['PredictedSummary'],
                        references=df['ExpectedSummary'])
print(results)


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:835: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


{'rouge1': 0.39647615250842394, 'rouge2': 0.19129789457120366, 'rougeL': 0.21648069005226922, 'rougeLsum': 0.34956392016782867}


In [ ]:
# Save DataFrame to CSV
df.to_csv('/content/gdrive/MyDrive/W266 Final Project/output/LongT5_1000_result.csv', index=False)

In [ ]:
df['ExpectedSummary'][0]

'Civil Appeal No. 2828 of 1977. Appeal by Special leave from the Judgment and order dated the 7th September, 1976 of the Punjab & Haryana High Court in Letters Patent Appeal No. 352 of 1975. Jawahar Lal Gupta, Janendralal and B.R. Agarwala for the Appellant. Randhir Jain for the Respondent. The Judgment of the Court Was delivered by VARADARAJAN, J. This appeal by special leave is by Punjab University against the Judgment of a Full Bench of the Punjab and Haryana High Court in the Letters Patent Appeal 352 of 1975 confirming the judgment of a learned Single Judge in W.P. 1017 of 1975. Subash Chander, respondent 1 in this appeal, joined the Daya Nand Medical College, Ludhiana, for the M.B.B.S. course in 1965 when Regulation 25 of the Punjab University was in force. That regulation required a minimum of 50 per cent of marks to pass in each subject. However Rule 7.1 relating to the M.B.B.S, and certain other courses provided that "a candidate who fails in one or more papers/subjects and/or

### LEDForConditionalGeneration

In [ ]:
import torch
from transformers import AutoTokenizer, LEDForConditionalGeneration

model = LEDForConditionalGeneration.from_pretrained("allenai/led-large-16384-arxiv")
tokenizer = AutoTokenizer.from_pretrained("allenai/led-large-16384-arxiv")

In [ ]:
len(baseline_output[0])

100

In [ ]:
inputs = 'summarize: ' + str(baseline_output[0])
# Tokenize the input text
input_ids = tokenizer.encode(inputs, return_tensors="pt", max_length=16384, truncation=True)

# global attention on the first token
# global_attention_mask = torch.zeros_like(input_ids)
# global_attention_mask[:, 0] = 1

# Generate summary using the model
summary_ids = model.generate(input_ids, num_beams=3, max_length=1024, early_stopping=True)

# Decode the generated summary IDs back to text
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

pprint(summary, compact = True)

(' this is an account of the proceedings which led to an order for financial '
 'relief after a marriage had been dissolved ( or annulled ) in a foreign '
 'country.                                                                                                                                                                                                                              ')


In [ ]:
rouge = evaluate.load('rouge')
predictions = summary
references = train_df_filter.iloc[0]['summary']
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)

ValueError: ignored

In [8]:
len(train_data_original['judgement'][0].split())
# len(extractive_output["Summary"][207].split())

10327

In [10]:
# Assuming 'extractive_output["Summary"]' is the list containing summaries

# Step 1: Calculate word count for each record in the column
word_counts = []

for summary in train_data_original["judgement"]:
    if isinstance(summary, str):  # Check if the summary is a string
        word_counts.append(len(summary.split()))

# Step 2: Sum all the word counts
total_word_count = sum(word_counts)

# Step 3: Divide the sum by the total number of valid records to get the average word count
if len(word_counts) > 0:
    average_word_count = total_word_count / len(word_counts)
    print("Average word count:", average_word_count)
else:
    print("No valid summaries found.")


Average word count: 5275.510034960507


### Windowing technic

#### original train data

In [9]:
# Initialize lists to store the summaries and expected summaries
predicted_summaries = []
expected_summaries = []
document_names = []

for index, row in train_data_original.head(1).iterrows():
    name = row['index']
    original_summary = row['summary']
    extractive_summary = row['judgement']

    # Divide the extractive summary into three parts
    summary_parts = [extractive_summary[i:i+int(len(extractive_summary)/4)] for i in range(0, len(extractive_summary), int(len(extractive_summary)/4))]

    # Initialize a variable to store combined summaries
    combined_summary = ""

    # Generate summaries for each part and combine them
    for part in summary_parts:
        ARTICLE_TO_SUMMARIZE = "summarize: " + part

        # Tokenize the article
        inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=4096, return_tensors="pt", truncation=True, padding=True)
        input_ids = inputs.input_ids.to("cuda")
        attention_mask = inputs.attention_mask.to("cuda")

        # Generate the summary
        outputs = model.generate(input_ids, attention_mask=attention_mask, num_beams=3, min_length=125, max_length=256, temperature=1.0, repetition_penalty=2.5, length_penalty=2.0)
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Append the generated summary to the combined summary
        combined_summary += summary

    # Append the predicted and expected summaries to their respective lists
    predicted_summaries.append(combined_summary)
    expected_summaries.append(original_summary)
    document_names.append(name)

# Creating a dictionary to represent the data
data = {
    'DocumentName': document_names,
    'ExpectedSummary': expected_summaries,
    'PredictedSummary': predicted_summaries
}

# Creating the DataFrame
df = pd.DataFrame(data)

# Perform Rouge evaluation on the whole dataset
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=df['PredictedSummary'],
                        references=df['ExpectedSummary'])
print(results)


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:835: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


{'rouge1': 0.5204200700116687, 'rouge2': 0.1985981308411215, 'rougeL': 0.22287047841306884, 'rougeLsum': 0.43873978996499413}


#### try with 5 records first

In [6]:
# Initialize lists to store the summaries and expected summaries
predicted_summaries = []
expected_summaries = []
document_names = []

for index, row in example_df.iterrows():
    name = row['Index']
    original_summary = row['Summary']
    extractive_summary = row['ExtractiveSummary']

    # Divide the extractive summary into three parts
    summary_parts = [extractive_summary[i:i+int(len(extractive_summary)/4)] for i in range(0, len(extractive_summary), int(len(extractive_summary)/4))]

    # Initialize a variable to store combined summaries
    combined_summary = ""

    # Generate summaries for each part and combine them
    for part in summary_parts:
        ARTICLE_TO_SUMMARIZE = "summarize: " + part

        # Tokenize the article
        inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=4096, return_tensors="pt", truncation=True, padding=True)
        input_ids = inputs.input_ids.to("cuda")
        attention_mask = inputs.attention_mask.to("cuda")

        # Generate the summary
        outputs = model.generate(input_ids, attention_mask=attention_mask, num_beams=3, min_length=125, max_length=256, temperature=1.0, repetition_penalty=2.5, length_penalty=2.0)
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Append the generated summary to the combined summary
        combined_summary += summary

    # Append the predicted and expected summaries to their respective lists
    predicted_summaries.append(combined_summary)
    expected_summaries.append(original_summary)
    document_names.append(name)

# Creating a dictionary to represent the data
data = {
    'DocumentName': document_names,
    'ExpectedSummary': expected_summaries,
    'PredictedSummary': predicted_summaries
}

# Creating the DataFrame
df = pd.DataFrame(data)

# Perform Rouge evaluation on the whole dataset
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=df['PredictedSummary'],
                        references=df['ExpectedSummary'])
print(results)


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:835: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


{'rouge1': 0.5210855575122857, 'rouge2': 0.23674113879840997, 'rougeL': 0.25956815793960336, 'rougeLsum': 0.44103068607430773}


#### run the whole 1000 records

In [7]:
# Initialize lists to store the summaries and expected summaries
predicted_summaries = []
expected_summaries = []
document_names = []

for index, row in extractive_output.iterrows():
    name = row['Index']
    original_summary = row['Summary']
    extractive_summary = row['ExtractiveSummary']

    # Divide the extractive summary into three parts
    summary_parts = [extractive_summary[i:i+int(len(extractive_summary)/4)] for i in range(0, len(extractive_summary), int(len(extractive_summary)/4))]

    # Initialize a variable to store combined summaries
    combined_summary = ""

    # Generate summaries for each part and combine them
    for part in summary_parts:
        ARTICLE_TO_SUMMARIZE = "summarize: " + part

        # Tokenize the article
        inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=4096, return_tensors="pt", truncation=True, padding=True)
        input_ids = inputs.input_ids.to("cuda")
        attention_mask = inputs.attention_mask.to("cuda")

        # Generate the summary
        outputs = model.generate(input_ids, attention_mask=attention_mask, num_beams=3, min_length=125, max_length=256, temperature=1.0, repetition_penalty=2.5, length_penalty=2.0)
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Append the generated summary to the combined summary
        combined_summary += summary

    # Append the predicted and expected summaries to their respective lists
    predicted_summaries.append(combined_summary)
    expected_summaries.append(original_summary)
    document_names.append(name)

# Creating a dictionary to represent the data
data = {
    'DocumentName': document_names,
    'ExpectedSummary': expected_summaries,
    'PredictedSummary': predicted_summaries
}

# Creating the DataFrame
df = pd.DataFrame(data)

# Perform Rouge evaluation on the whole dataset
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=df['PredictedSummary'],
                        references=df['ExpectedSummary'])
print(results)


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:835: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


{'rouge1': 0.4781344807734289, 'rouge2': 0.23163645291921503, 'rougeL': 0.24573294957117503, 'rougeLsum': 0.41552881834840816}


In [8]:
# Save DataFrame to CSV
df.to_csv('/content/gdrive/MyDrive/W266 Final Project/output/LongT5_1000_4foldwindow_result.csv', index=False)


In [ ]:
df

,DocumentName,ExpectedSummary,PredictedSummary
0,4809.txt,The first respondent joined M.B.B.S. course of...,this is a case report of a graduate graduate w...
1,543.txt,"On the death of R, a Hindu jat, in April or Ma...",this is a case report of a woman who died in 1...
2,372.txt,An application was filed by the first responde...,"in this article, we present the case of an app..."
3,uksc-2010-0177.txt,Scottish Widows Plc (Scottish Widows) is a lif...,this is an appeal from an interlocutor of the ...
4,6391.txt,The appellant and the respondents applied for ...,the Chancellor of the Lucknow university while...


## Fine tuning LED

In [ ]:
# split data into training and validation dataset:
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
# Here, test_size represents the percentage of data to be used for the validation set
# You can adjust test_size according to your requirement (e.g., test_size=0.2 for 20% validation data)
train_df, val_df = train_test_split(extractive_output, test_size=0.2, random_state=42)

# Print the shapes of the resulting datasets
print("Training data shape:", train_df.shape)
print("Validation data shape:", val_df.shape)


Training data shape: (6178, 3)
Validation data shape: (1545, 3)
